In [2]:
import condb as con
import hashlib
import pickle

LOGIN PAGE

In [6]:
# EXAMPLE USER INPUT
input_username = 'oil'
input_password = 'oil1234'

# HASH PASSWORD
input_hash_pass = hashlib.sha1(input_password.encode())
input_hash_pass = input_hash_pass.hexdigest()

# GET USER DATA FROM DATABASE
user_data = con.selectData(table='users', condition=f"user_name='{input_username}'")

# CONDITION
if len(user_data) >= 1:
    user_passwors = user_data[0][2]
    if user_passwors == input_hash_pass:
        print('login success')
    else:
        print('password incorrect!')
else:
    print('username incorrect!')

login success


REGISTER PAGE

In [7]:
# EXAMPLE USER INPUT
input_username = 'manow'
input_password = 'manow1234'
input_phone_number = '0987654321'
input_email = 'manow.y@kkumail.com'
input_bank_name = 'kasikorn'
input_bank_number = '1234567890'
input_address = 'near by khon kaen airport'

# HASH PASSWORD
input_hash_pass = hashlib.sha1(input_password.encode())
input_hash_pass = input_hash_pass.hexdigest()

# GET USER DATA FROM DATABASE
find_user = con.selectData(table='users', condition=f"user_name='{input_username}'")

# FORMAT INPUT DATA FOR INSERT
insert_data = f"""  '{input_username}', 
                    '{input_hash_pass}', 
                    '{input_phone_number}', 
                    '{input_email}', 
                    '{input_address}', 
                    'user', 
                    '{input_bank_number}', 
                    '{input_bank_name}'
                """

# FIND MAX OF PRIMARY KRY FROM USERS TABLE
max_primary = con.selectData(table='users', column='max(user_id)')
max_primary = max_primary[0][0]

# CONDITION
if len(find_user) > 0:
    print(f'{input_username} already used!')
else:
    con.insertData(table='users', primary_key=max_primary+1, values=insert_data)
    print('register complete')

users 1 row inserted
register complete


SALE PAGE

In [7]:
# AI PART
# LOAD AI MODEL
with open(r'D:\KKU_World\1_2\DBMS\termProject\python\Ai\random_forest_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)
ai_model = loaded_model

#EXAMPLE USER INPUT
input_model = 'EcoSport'
input_produc_year = '2020'
input_engine_size = '4.0'
input_transmission = 'Manual'
input_fuel = 'Diesel'
input_mile_per_gal = '35'
input_mile_used = '16890'

# MAPPING STRING FROM USER INPUT TO NUMBER FOR AI
model_mapping = {
    'Fiesta': 0,
    'Focus': 1,
    'Kuga': 2,
    'EcoSport': 3,
    'etc.': 4-22
}
transmission_mapping = {
    'Manual': 0,
    'Automatic': 1,
    'Semi-Auto': 2
}
fuel_mapping = {
    'Petrol': 0,
    'Diesel': 1,
    'Hybrid': 2,
    'Electric': 3,
    'Other': 4
}

# FORMAT DATA FOR AI
format_data = [ int(model_mapping[input_model]), 
                int(input_produc_year), 
                int(transmission_mapping[input_transmission]), 
                float(input_mile_used), 
                int(fuel_mapping[input_fuel]), 
                float(input_mile_per_gal), 
                float(input_engine_size)
            ]

# PREDICT PRICE
ai_offer_price = ai_model.predict([format_data])
ai_offer_price = int(ai_offer_price[0])
print('Ai offer', ai_offer_price, 'dollar\n')

Ai offer 29395 dollar



c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [9]:
# DATABASE PART
# EXAMPLE DATA
user_id = 2
file_name = 'fordCar_002.png'
input_model = 'EcoSport'
input_license_plate = 'PKU-2942'
input_produc_year = '2020'
input_engine_size = '4.0'
input_transmission = 'Manual'
input_fuel = 'Diesel'
input_mile_per_gal = '35'
input_mile_used = '16890'
input_user_offer_price = '30000'

# FIND MAX OF PRIMARY KEY FROM SALES_HISTORY TABLE
max_sale_his_primary = con.selectData(table='sales_history', 
                                        column='max(sale_id)')
max_sale_his_primary = max_sale_his_primary[0][0]

# FUNCTION FIND EXIST CAR ID FROM CAR TABLE
    # if found car in table return car id
    # if not found insert new car and return new car id
def find_car_id(model,  production_year, engine_size, miles_per_gallon, trans_id, fuel_id):
    """
        if found exist car in DB return exist car id
        if not found insert new car and return new car id
    """
    condition_find_car = f"""model='{model}' 
                            and production_year={production_year} 
                            and engine_size={engine_size} 
                            and miles_per_gallon={miles_per_gallon} 
                            and trans_id={trans_id} and fuel_id={fuel_id}
                            """
    
    exist_car = con.selectData(table='car', condition=condition_find_car)
    
    if len(exist_car) <= 0:
        format_insert_data = f"'{model}', {production_year}, {engine_size}, {miles_per_gallon}, {trans_id}, {fuel_id}"
        max_car_id = con.selectData(table='car', column='max(car_id)')
        max_car_id = max_car_id[0][0]
        
        if max_car_id == None:
            max_car_id = 0
        con.insertData(table='car', primary_key=max_car_id+1, values=format_insert_data)
        return max_car_id+1
    else:
        return exist_car[0][0]

# FIND CAR ID FOR INSERT DATA AND FORMAT INSERT DATA
car_id = find_car_id(input_model, 
                    input_produc_year, 
                    input_engine_size, 
                    input_mile_per_gal, 
                    transmission_mapping[input_transmission], 
                    fuel_mapping[input_fuel]
                    )
format_insert_data = f"""
                    '{user_id}', 
                    'user offer', 
                    '{file_name}', 
                    '{input_mile_used}', 
                    '{ai_offer_price}', 
                    '{input_user_offer_price}', 
                    null, 
                    '{input_license_plate}', 
                    '{car_id}'"""

con.insertData(table='sales_history', 
                primary_key=max_sale_his_primary+1, 
                values=format_insert_data)
print('✅ Data saved please wait response from admin')

sales_history 1 row inserted
✅ Data saved please wait response from admin


USER SALE HISTORY PAGE

In [3]:
# EXAMPLE USER DATA
user_id = 1

# DEFINE QUERY STRING
query_str = f"""
        SELECT path_picture, model, production_year, mile_used, license_plate, predict_price, user_offer_price, admin_offer_price, status
        FROM sales_history, car
        WHERE sales_history.car_id = car.car_id and sales_history.user_id = {user_id}
        ORDER BY sale_id
    """

sales_history_data = con.querySql(query_str)

for i in sales_history_data:
    print(sales_history_data)

[('oil_7.png', 'EcoSport', 2020, 16890, 'PKU-2942', 29395, 30000, None, 'user offer')]


ADMIN SALE HISTORY PAGE

In [11]:
# GET ALL SALES HISTORY DATA FROM SALES_HISTORY TABLE
sales_data = con.selectData(table='admin_sale_history_data')
for i in sales_data:
    print(i)

('oil_7.png', 1, 'oil', 'EcoSport', 2020, 16890, 'PKU-2942', '0984321907', 29395, 30000, None, 'user offer')
('god_8.png', 3, 'god', 'Puma', 2019, 31650, 'KPC-1067', '0984321907', 30048, None, None, 'user confirm')
('manow_9.png', 4, 'manow', 'Fiesta', 2023, 24926, 'KKU-2566', '0987654321', 29282, 29500, 29300, 'user offer')


In [12]:
# EXAMPLE DATA FOR UPDATE SALES STATUS
user_id = 4
sale_id = 9
admin_offer_price = 29300

con.updateData(table='sales_history', 
                set=f'admin_offer_price={admin_offer_price}', 
                condition=f"user_id={user_id} and sale_id={sale_id}")

sales_history 1 row updated


True